# Code generator

Neural network that generates Golang code

In [25]:
import json
import numpy as np
import os

In [28]:
class Dataset(object):
    dataset_file = 'dataset.dt'
    dataset_folder = 'dataset'
    
    def __init__(self):
        dataset = self.load_dataset()
        self.encoded, self.vocab_to_int, self.int_to_vocab = dataset
        
    def load_dataset(self):
        #if self.exists_clean_dataset():
         #   return self.load_clean_dataset()
        dataset = self.create_dataset()
        self.save_clean_dataset(dataset)
        return dataset

    def exists_clean_dataset(self):
        return os.path.isfile(self.dataset_file)

    def load_clean_dataset(self):
        with open(self.dataset_file, "r") as d:
            dataset = json.loads(d.read())
            return (
                np.array(dataset["encoded"]),
                dataset["vocab_to_int"],
                dataset["int_to_vocab"]
            )
        
    def create_dataset(self):
        file_names = self.get_file_names()
        content = self.get_content_from_files(file_names)
        
    
    
    def get_file_names(self):
        files = os.listdir(self.dataset_folder)
        return [f for f in files if f.endswith('.go')]
    
    def get_content_from_files(self, file_names):
        content = ''
        for name in file_names:
            with open(os.path.join(self.dataset_folder, name), 'r') as f:
                content += f.read + '\n'
    
    def save_clean_dataset(self, dataset):
        encoded, v_to_int, int_to_v = dataset
        with open(self.dataset_file, "w+") as d:
            d.write(json.dumps({
                'encoded' : [int(i) for i in encoded], 
                'vocab_to_int' : v_to_int,
                'int_to_vocab' : int_to_v
            }))
    
dataset = Dataset()

['trieval.go' 'zsyscall_solaris_amd64.go' 'keysparsing.go' 'uint32.go'
 'zerrors_freebsd_amd64.go' 'weighter_test.go' 'syscall_darwin.go'
 'logger_bench_test.go' 'mapstructure_benchmark_test.go' 'builder_test.go'
 'golangflag_test.go' 'profile_test.go' 'punycode.go' 'enctest.go'
 'basic.go' 'ucd_test.go' 'logger.go' 'syslog.go' 'bidi.go'
 'syscall_linux_amd64_gc.go' 'go1_2.go' 'internal.go' 'col.go'
 'hzgb2312.go' 'zsyscall_dragonfly_amd64.go' 'colelem.go'
 'zsyscall_linux_mipsle.go' 'config.go' 'ipnet.go' 'flock.go'
 'types_openbsd.go' 'models.go' 'merge_test.go' 'zsyscall_freebsd_arm.go'
 'old_test.go' 'zerrors_linux_arm64.go' 'afero.go' 'zsysnum_openbsd_386.go'
 'int_slice_test.go' 'runenames.go' 'int_slice.go'
 'type_specific_codegen.go' 'helpers_unix_test.go' 'parser.go'
 'cldr_test.go' 'assertion_forward.go' 'export_test.go' 'beep.go'
 'syscall_linux_arm64.go' 'default_notepad.go' 'exported.go'
 'endian_little.go' 'inotify_poller_test.go' 'icu.go' 'display.go'
 'doc_test.go' 'db.

TypeError: 'NoneType' object is not iterable

In [1]:
class Model(object):
    pass    

In [4]:
class Trainer(Model):
    pass

In [5]:
class Predcitor(Model):
    pass